## MG-GY 8401: Programming for Business Intelligence and Analytics
### Lab 6

We will take a look at support for text processing in the `pandas` package. Along the way, we will get experience with regular expressions in the `re` package.

In [3]:
# import some packages

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

import re

# change some settings

pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', 10)
plt.rcParams['figure.figsize'] = (10,8)

# indicate path to data 

import os
path_to_dataset = os.environ["HOME"] + "/shared/lecture-6/nyc_restaurant_inspections.csv"
path_to_dataset_raw = os.environ["HOME"] + "/shared/lab-6/nyc_restaurant_inspections_raw.csv"

In Lecture 6, we studied data from the New York City Department of Health and Mental Hygiene. 

In [2]:
violations = pd.read_csv(path_to_dataset)
violations

,CAMIS,RECORD DATE,VIOLATION DESCRIPTION,SCORE,desc,is_clean,is_vermin,is_surface,is_human,is_permit
0,50018511,11/05/2018,Filth flies or food/refuse/sewage-associated (...,19.0,filth flies or food/refuse/sewage-associated f...,0,0,0,0,0
1,50012427,11/05/2018,Non-food contact surface improperly constructe...,19.0,non-food contact surface improperly constructe...,1,0,1,0,0
2,50075209,11/05/2018,Personal cleanliness inadequate. Outer garment...,13.0,personal cleanliness inadequate. outer garment...,1,0,0,1,0
3,50001110,11/05/2018,Non-food contact surface improperly constructe...,27.0,non-food contact surface improperly constructe...,1,0,1,0,0
4,41166868,11/05/2018,"Food contact surface not properly washed, rins...",13.0,"food contact surface not properly washed, rins...",1,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...
19087,50071777,11/05/2018,Proper sanitization not provided for utensil w...,13.0,proper sanitization not provided for utensil w...,1,0,0,0,0
19088,41674774,11/05/2018,Filth flies or food/refuse/sewage-associated (...,16.0,filth flies or food/refuse/sewage-associated f...,0,0,0,0,0
19089,50044176,11/05/2018,Facility not vermin proof. Harborage or condit...,13.0,facility not vermin proof. harborage or condit...,0,1,0,0,0
19090,50039443,11/05/2018,Evidence of mice or live mice present in facil...,33.0,evidence of mice or live mice present in facil...,0,0,0,0,0


From the Department of Health and Mental Hygience repository, we have a sample of 19092 restaurant inspections. 

In [6]:
violations.columns

Index(['CAMIS', 'DBA', 'BORO', 'BUILDING', 'STREET', 'ZIPCODE', 'PHONE',
       'CUISINE DESCRIPTION', 'INSPECTION DATE', 'ACTION', 'VIOLATION CODE',
       'VIOLATION DESCRIPTION', 'CRITICAL FLAG', 'SCORE', 'GRADE',
       'GRADE DATE', 'RECORD DATE', 'INSPECTION TYPE'],
      dtype='object')

We have 10 columns

- `CAMIS` : unique identifier of the restaurant 
- `RECORD DATE` : date of inspection 
- `VIOLATION DESCRIPTION` : description of violations
- `SCORE` : score of the restaurant following health inspection
- `desc` : detailed description of violation 
- `is_clean`, `is_vermin`, `is_surface`, `is_human`, `is_permit`
 * type of violation 
 
Note that we have processed the raw dataset from the Department of Health and Mental Hygience repository. 

In [7]:
violations_raw = pd.read_csv(path_to_dataset_raw)
violations_raw.head(3)

NameError: name 'path_to_dataset_raw' is not defined

The raw dataset has 18 columns.

In [9]:
violations_raw.columns

Index(['CAMIS', 'DBA', 'BORO', 'BUILDING', 'STREET', 'ZIPCODE', 'PHONE',
       'CUISINE DESCRIPTION', 'INSPECTION DATE', 'ACTION', 'VIOLATION CODE',
       'VIOLATION DESCRIPTION', 'CRITICAL FLAG', 'SCORE', 'GRADE',
       'GRADE DATE', 'RECORD DATE', 'INSPECTION TYPE'],
      dtype='object')

Note that we lack the columns 

- `is_clean`
- `is_vermin`
- `is_surface`
- `is_human`
- `is_permit`

We need to process the text in the `VIOLATION DESCRIPTION` column to determine the 5 types of violations. In this lab, we want to study text as data. We will explore string processing to understand the extraction of information from text like `VIOLATION DESCRIPTION`.

### Working with Strings

Suppose we want to join the follow two tables. 

In [8]:
county_and_state = pd.read_csv("county_and_state.csv")
county_and_state

,County,State
0,De Witt County,IL
1,Lac qui Parle County,MN
2,Lewis and Clark County,MT
3,St John the Baptist Parish,LS


Here we have state and county.

In [9]:
county_and_pop = pd.read_csv("county_and_population.csv")    
county_and_pop

,County,Population
0,DeWitt,16798
1,Lac Qui Parle,8067
2,Lewis & Clark,55716
3,St. John the Baptist,43044


Here we have county and population.

We want to link the records through the `County` column. However, we have mismatches in the data. For example, we cannot link `Lewis & Clark` and `Lewis and Clark County`.

In [10]:
def standardize_county(county_name):
    return (
        county_name
        .lower()               # lower case
        .replace(' ', '')      # remove spaces
        .replace('&', 'and')   # replace &
        .replace('.', '')      # remove dot
        .replace('county', '') # remove county
        .replace('parish', '') # remove parish
    )

Before we can join the table, we need to standardize the strings. We want to convert many possible representations into a standard format.

In [11]:
clean_county = []

for county in county_and_pop['County']:
    clean_county.append(standardize_county(county))
    
county_and_pop['clean_county'] = clean_county

Instead of using a loop, we can use the `pandas` function `map`. 

In [12]:
county_and_state['clean_county'] = county_and_state['County'].map(standardize_county)

Now we can join the tables.

In [13]:
pd.merge(left = county_and_pop,
         right = county_and_state,
         left_on = 'clean_county', 
         right_on = 'clean_county')

,County_x,Population,clean_county,County_y,State
0,DeWitt,16798,dewitt,De Witt County,IL
1,Lac Qui Parle,8067,lacquiparle,Lac qui Parle County,MN
2,Lewis & Clark,55716,lewisandclark,Lewis and Clark County,MT
3,St. John the Baptist,43044,stjohnthebaptist,St John the Baptist Parish,LS


### Matching Patterns

We tend to work with structured data in a tabular format consisting of rows and columns. However, we might have to work with unstructured data. Text is a common source of unstructured data. Note that the file extension `txt` can denote a text file without any format.

In [14]:
connection_to_file = open('log.txt', 'r')

log_lines = connection_to_file.readlines()

connection_to_file.close()

We took three steps to reading the data in the text file. 

1. We have used the Python function `open` to establish a connection to a file. Note `r` indicates that we will read from the file. 
1. We use the function `readlines` to read the data. The function converst each line of the file to a string stored in a list.
1. We close the connection to the file.

An alternative approach to connecting and disconnecting to files is the keyword `with`.

In [15]:
with open('log.txt', 'r') as connection_to_file:
    log_lines = connection_to_file.readlines()

Here we establish the connection to the file in the scope of the `with` statement. Following the `with` statement the variable `connection_to_file` goes out of scope which closes the connection to the file.

In [16]:
log_lines

['169.237.46.168 - - [26/Jan/2014:10:47:58 -0800] "GET /stat141/Winter04/ HTTP/1.1" 200 2585 "http://anson.nyu.edu/courses/"\n',
 '193.205.203.3 - - [2/Feb/2005:17:23:6 -0800] "GET /stat141/Notes/dim.html HTTP/1.0" 404 302 "http://eeyore.nyu.edu/stat141/Notes/session.html"\n',
 '169.237.46.240 - "" [3/Feb/2006:10:18:37 -0800] "GET /stat141/homework/Solutions/hw1Sol.pdf HTTP/1.1"']

Suppose we want to extract the day, month, year, hour, minutes, seconds, and timezone. Note that these items are not in a fixed position in the string. 

In [17]:
log_lines[0][20:31]

'26/Jan/2014'

By slicing from character 20 to character 31, we can extract the date from the first line.

In [18]:
log_lines[1][20:31]

'/Feb/2005:1'

However, the date appears at a different position in the second line. So we cannot extract characters based on location in the text. 

We could try to use the `split` function that splits a string into a list of strings.

In [19]:
first = log_lines[0]
first

'169.237.46.168 - - [26/Jan/2014:10:47:58 -0800] "GET /stat141/Winter04/ HTTP/1.1" 200 2585 "http://anson.nyu.edu/courses/"\n'

From the first lines of text, we need to extract the date and time. We can try to split at brackets, slash, colon and whitespace.

In [20]:
pertinent = first.split("[")[1].split(']')[0]
day, month, rest = pertinent.split('/')
year, hour, minute, rest = rest.split(':')
seconds, time_zone = rest.split(' ')
day, month, year, hour, minute, seconds, time_zone

('26', 'Jan', '2014', '10', '47', '58', '-0800')

While we manage to extract the date and time, we need to repeatedly split the string. Instead we can use regular expresions.

In [21]:
pattern = r'\[(\d+)/(\w+)/(\d+):(\d+):(\d+):(\d+) (.+)\]'
re.findall(pattern, first)

[('26', 'Jan', '2014', '10', '47', '58', '-0800')]

### Processing Strings in Tables

We want to extract different types of violations from the `violations_raw`.

In [22]:
violations_raw = violations_raw.copy()[['CAMIS', 'RECORD DATE', 'VIOLATION DESCRIPTION', "SCORE"]]

NameError: name 'violations_raw' is not defined

We observe that the `VIOLATION DESCRIPTION` columns lacks a format. 

In [5]:
violations_raw["VIOLATION DESCRIPTION"].value_counts()

Non-food contact surface improperly constructed. Unacceptable material used. Non-food contact surface or equipment improperly maintained and/or not properly sealed, raised, spaced or movable to allow accessibility for cleaning on all sides, above and underneath the unit.    2637
Facility not vermin proof. Harborage or conditions conducive to attracting vermin to the premises and/or allowing vermin to exist.                                                                                                                                                 2091
Evidence of mice or live mice present in facility's food and/or non-food areas.                                                                                                                                                                                                    1335
Food not protected from potential source of contamination during storage, preparation, transportation, display or service.                                      

We need to standardize the text. If we have a column consisting of strings, then we can use the `pandas` functions available in `str`.

In [6]:
violations_raw['VIOLATION DESCRIPTION Standardized'] = (violations_raw['VIOLATION DESCRIPTION']
             .str.replace(r'(\(|\))', '')
             .str.strip()
             .str.lower())

violations_raw.head()

,CAMIS,RECORD DATE,VIOLATION DESCRIPTION,SCORE,VIOLATION DESCRIPTION Standardized
0,50018511,11/05/2018,Filth flies or food/refuse/sewage-associated (...,19.0,filth flies or food/refuse/sewage-associated f...
1,50012427,11/05/2018,Non-food contact surface improperly constructe...,19.0,non-food contact surface improperly constructe...
2,50075209,11/05/2018,Personal cleanliness inadequate. Outer garment...,13.0,personal cleanliness inadequate. outer garment...
3,50001110,11/05/2018,Non-food contact surface improperly constructe...,27.0,non-food contact surface improperly constructe...
4,41166868,11/05/2018,"Food contact surface not properly washed, rins...",13.0,"food contact surface not properly washed, rins..."


Here we replace parentheses with white-space, remove starting and ending white-space, and convert capital letters to lowercase letters.

In [8]:
violations_raw['VIOLATION DESCRIPTION Standardized'].value_counts()

non-food contact surface improperly constructed. unacceptable material used. non-food contact surface or equipment improperly maintained and/or not properly sealed, raised, spaced or movable to allow accessibility for cleaning on all sides, above and underneath the unit.    2637
facility not vermin proof. harborage or conditions conducive to attracting vermin to the premises and/or allowing vermin to exist.                                                                                                                                                 2091
evidence of mice or live mice present in facility's food and/or non-food areas.                                                                                                                                                                                                    1335
food not protected from potential source of contamination during storage, preparation, transportation, display or service.                                      

Note we have not changed the number of distinct descriptions. Despite the standardization, we have 81 different descriptions. 

We have 408 missing values in the `VIOLATION DESCRIPTION Standardized` column. We fill with a blank string.

In [11]:
violations_raw["VIOLATION DESCRIPTION Standardized"] = violations_raw["VIOLATION DESCRIPTION Standardized"].fillna(" ")

We can try to use `pandas` function `str.contains` to extract keywords from the text with regular expressions.

In [17]:
violations_raw["is_clean"] = violations_raw['VIOLATION DESCRIPTION Standardized'].str.contains(r'clean|sanit')

violations_raw["is_vermin"] = violations_raw['VIOLATION DESCRIPTION Standardized'].str.contains(r'vermin')

violations_raw["is_surface"] =  violations_raw['VIOLATION DESCRIPTION Standardized'].str.contains(r'wall|ceiling|floor|surface')

violations_raw["is_human"] = violations_raw['VIOLATION DESCRIPTION Standardized'].str.contains(r'hand|glove|hair|nail')

violations_raw["is_permit"] = violations_raw['VIOLATION DESCRIPTION Standardized'].str.contains(r'permit|certif')

Since we have boolean values, we should convert to integers.

In [19]:
for header in [ 'is_clean', 'is_vermin', 'is_surface', 'is_human', 'is_permit']:
    violations_raw[header] = violations_raw[header].astype(int)

Now we have the same columns in `violations`.

In [21]:
violations_raw.head()

,CAMIS,RECORD DATE,VIOLATION DESCRIPTION,SCORE,VIOLATION DESCRIPTION Standardized,...,is_clean,is_vermin,is_surface,is_human,is_permit
0,50018511,11/05/2018,Filth flies or food/refuse/sewage-associated (...,19.0,filth flies or food/refuse/sewage-associated f...,...,0,0,0,0,0
1,50012427,11/05/2018,Non-food contact surface improperly constructe...,19.0,non-food contact surface improperly constructe...,...,1,0,1,0,0
2,50075209,11/05/2018,Personal cleanliness inadequate. Outer garment...,13.0,personal cleanliness inadequate. outer garment...,...,1,0,0,1,0
3,50001110,11/05/2018,Non-food contact surface improperly constructe...,27.0,non-food contact surface improperly constructe...,...,1,0,1,0,0
4,41166868,11/05/2018,"Food contact surface not properly washed, rins...",13.0,"food contact surface not properly washed, rins...",...,1,0,1,0,0


In [4]:
re.findall(r"[A-Za-z][a-z]*", "camelCase")

['camel', 'Case']

In [ ]:
re.findall(r"[A-Za-z][a-z]*", "sidfzdnsi")